# Make ToO commands for satelites


In [ ]:
import pandas as pd
import re
from datetime import datetime

# Static dictionary with fixed ToO command parameters
static_params = {
    'filt': 'r',
    'exp': '10',
    'noexp': '1',
    'p': '7000',
    'obs_type': 'NS',
    'domesleep': '50',
    'iit_obs': 'sameer',
    'hanle_obs': 'rigzin',
    'stack': 'N',
    'propnums': 'P07'
    
}

# Template command from maketoo_template
template_command = "sudo python make_too_v2.py -ra 'PH_RIGHT_ASC' -dec 'PH_DECLINATION' -ra_rate 'PH_RATE_RA' -dec_rate 'PH_RATE_DEC' -obs_time_utc 'PH_OBS_TIME_UTC' -filt 'PH_FILT' -exp 'PH_EXPOSURE' -noexp '1' -name 'PH_NAME' -p 'PH_PRIORITY' -obs_type 'PH_OBS_TYPE' -domesleep '50' -iit_obs 'PH_IIT_OBS' -hanle_obs 'PH_HANLE_OBS' -stack 'N' -propnums 'P07'"

print("Static parameters dictionary:")
for key, value in static_params.items():
    print(f"  {key}: {value}")
print(f"\nTemplate command:\n{template_command}")


Static parameters dictionary:
  filt: r
  exp: 10
  noexp: 1
  p: 7000
  obs_type: NS
  domesleep: 50
  iit_obs: sameer
  hanle_obs: rigzin
  stack: N
  propnums: P07

Template command:
sudo python make_too.py -ra 'PH_RIGHT_ASC' -dec 'PH_DECLINATION' -ra_rate 'PH_RATE_RA' -dec_rate 'PH_RATE_DEC' -obs_time_utc 'PH_OBS_TIME_UTC' -filt 'PH_FILT' -exp 'PH_EXPOSURE' -noexp '1' -name 'PH_NAME' -p 'PH_PRIORITY' -obs_type 'PH_OBS_TYPE' -domesleep '50' -d 'PH_DATE' -iit_obs 'PH_IIT_OBS' -hanle_obs 'PH_HANLE_OBS' -stack 'N' -propnums 'P07'


In [44]:
def parse_csv_ephemeris(csv_file_path, satellite_name):
    """
    Parse the CSV ephemeris file and extract observation data
    """
    observations = []
    
    with open(csv_file_path, 'r') as file:
        lines = file.readlines()
    
    # Find the data section (after $$SOE)
    data_started = False
    for line in lines:
        if '$$SOE' in line:
            data_started = True
            continue
        if '$$EOE' in line:
            break
            
        if data_started and line.strip():
            # Skip header lines and cut-off messages
            if 'Date__(UT)__HR:MN:SS' in line or 'Airmass Cut-off' in line or 'Daylight Cut-off' in line:
                continue
                
            # Parse data lines that start with a date
            if re.match(r'\s*\d{4}-\w{3}-\d{2}\s+\d{2}:\d{2}:\d{2}', line):
                parts = line.strip().split(',')
                if len(parts) >= 7:
                    try:
                        # Extract UTC time
                        utc_time_raw = parts[0].strip()
                        # Parse the current format: "2025-Oct-08 16:28:00"
                        dt = datetime.strptime(utc_time_raw, '%Y-%b-%d %H:%M:%S')
                        # Convert to ISO format: "2025-10-08T16:28:00"
                        utc_time = dt.strftime('%Y-%m-%dT%H:%M:%S')

                        # Extract RA and DEC
                        ra = float(parts[3].strip())
                        dec = float(parts[4].strip())
                        
                        # Extract rates (dRA*cosD and d(DEC)/dt) - these are in arcseconds per hour
                        ra_rate_raw = float(parts[5].strip())
                        dec_rate_raw = float(parts[6].strip())
                        
                        # Convert rates from arcseconds per hour to degrees per hour (divide by 3600)
                        ra_rate = ra_rate_raw / 3600.0
                        dec_rate = dec_rate_raw / 3600.0
                                                
                        observations.append({
                            'utc_time': utc_time,
                            'ra': ra,
                            'dec': dec,
                            'ra_rate': ra_rate,
                            'dec_rate': dec_rate,
                            'satellite': satellite_name
                        })
                        
                    except (ValueError, IndexError) as e:
                        print(f"Error parsing line: {line.strip()}")
                        print(f"Error: {e}")
                        continue
    
    return observations

"""
# Parse both CSV files
print("Parsing NOAA CSV file...")
noaa_obs = parse_csv_ephemeris('/Users/meghapatil/Documents/Sameer/Physics/vb/git/satelites/noaa.csv', 'NOAA_19')

print("Parsing NISAR CSV file...")
nisar_obs = parse_csv_ephemeris('/Users/meghapatil/Documents/Sameer/Physics/vb/git/satelites/ephem-nisar.csv', 'NISAR')

print(f"\nFound {len(noaa_obs)} NOAA observations")
print(f"Found {len(nisar_obs)} NISAR observations")

# Display first few observations for verification
print("\nFirst 3 NOAA observations:")
for i, obs in enumerate(noaa_obs[:3]):
    print(f"  {i+1}. {obs['utc_time']} - RA: {obs['ra']:.6f}, DEC: {obs['dec']:.6f}, RA_rate: {obs['ra_rate']:.6f}, DEC_rate: {obs['dec_rate']:.6f}")

print("\nFirst 3 NISAR observations:")
for i, obs in enumerate(nisar_obs[:3]):
    print(f"  {i+1}. {obs['utc_time']} - RA: {obs['ra']:.6f}, DEC: {obs['dec']:.6f}, RA_rate: {obs['ra_rate']:.6f}, DEC_rate: {obs['dec_rate']:.6f}")
"""


'\n# Parse both CSV files\nprint("Parsing NOAA CSV file...")\nnoaa_obs = parse_csv_ephemeris(\'/Users/meghapatil/Documents/Sameer/Physics/vb/git/satelites/noaa.csv\', \'NOAA_19\')\n\nprint("Parsing NISAR CSV file...")\nnisar_obs = parse_csv_ephemeris(\'/Users/meghapatil/Documents/Sameer/Physics/vb/git/satelites/ephem-nisar.csv\', \'NISAR\')\n\nprint(f"\nFound {len(noaa_obs)} NOAA observations")\nprint(f"Found {len(nisar_obs)} NISAR observations")\n\n# Display first few observations for verification\nprint("\nFirst 3 NOAA observations:")\nfor i, obs in enumerate(noaa_obs[:3]):\n    print(f"  {i+1}. {obs[\'utc_time\']} - RA: {obs[\'ra\']:.6f}, DEC: {obs[\'dec\']:.6f}, RA_rate: {obs[\'ra_rate\']:.6f}, DEC_rate: {obs[\'dec_rate\']:.6f}")\n\nprint("\nFirst 3 NISAR observations:")\nfor i, obs in enumerate(nisar_obs[:3]):\n    print(f"  {i+1}. {obs[\'utc_time\']} - RA: {obs[\'ra\']:.6f}, DEC: {obs[\'dec\']:.6f}, RA_rate: {obs[\'ra_rate\']:.6f}, DEC_rate: {obs[\'dec_rate\']:.6f}")\n'

In [45]:
def generate_too_command(observation, static_params, template_command):
    """
    Generate a ToO command by replacing placeholders with actual values
    """
    # Extract date from UTC time for the -d parameter
    utc_time_str = observation['utc_time']
    # Convert "2025-Oct-08 16:28:00" to "20251008"
    
    dt = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%S')
    date_str = dt.strftime('%Y%m%d')

    
    # Create the command by replacing placeholders
    command = template_command
    
    # Replace all placeholders
    replacements = {
        'PH_RIGHT_ASC': f"{observation['ra']:.8f}",
        'PH_DECLINATION': f"{observation['dec']:.8f}",
        'PH_RATE_RA': f"{observation['ra_rate']:.8f}",
        'PH_RATE_DEC': f"{observation['dec_rate']:.8f}",
        'PH_OBS_TIME_UTC': observation['utc_time'],
        'PH_FILT': static_params['filt'],
        'PH_EXPOSURE': static_params['exp'],
        'PH_NAME': observation['satellite'],
        'PH_PRIORITY': static_params['p'],
        'PH_OBS_TYPE': static_params['obs_type'],
        'PH_DATE': date_str,
        'PH_IIT_OBS': static_params['iit_obs'],
        'PH_HANLE_OBS': static_params['hanle_obs'],
        'PH_NO_EXP': static_params['noexp']
    }
    
    for placeholder, value in replacements.items():
        command = command.replace(placeholder, value)
    
    return command

# Generate commands for all observations
all_commands = []
"""
print("Generating ToO commands for NOAA observations...")
for i, obs in enumerate(noaa_obs):
    command = generate_too_command(obs, static_params, template_command)
    all_commands.append(command)
    print(f"NOAA Command {i+1}:")
    print(command)
    print()

print("Generating ToO commands for NISAR observations...")
for i, obs in enumerate(nisar_obs):
    command = generate_too_command(obs, static_params, template_command)
    all_commands.append(command)
    print(f"NISAR Command {i+1}:")
    print(command)
    print()

print(f"\nTotal commands generated: {len(all_commands)}")
print(f"NOAA commands: {len(noaa_obs)}")
print(f"NISAR commands: {len(nisar_obs)}")
"""
# Parse both CSV files
print("Parsing CSV file...")
allobs = parse_csv_ephemeris('satelites/RISAT2BR2.csv', 'RISAT2BR2')

print(f"\nFound {len(allobs)} observations")
# Display first few observations for verification
print("\nFirst 3 observations:")
for i, obs in enumerate(allobs[:3]):
    print(f"  {i+1}. {obs['utc_time']} - RA: {obs['ra']:.6f}, DEC: {obs['dec']:.6f}, RA_rate: {obs['ra_rate']:.6f}, DEC_rate: {obs['dec_rate']:.6f}")

print("Generating ToO commands for observations...")
for i, obs in enumerate(allobs):
    command = generate_too_command(obs, static_params, template_command)
    all_commands.append(command)
    #print(f"Obs Command {i+1}:")
    print(command)
    print()



Parsing CSV file...

Found 30 observations

First 3 observations:
  1. 2025-10-12T13:21:00 - RA: 291.813072, DEC: -12.734336, RA_rate: 1643.181944, DEC_rate: 592.812222
  2. 2025-10-12T13:22:00 - RA: 323.219882, DEC: -0.754622, RA_rate: 1938.500833, DEC_rate: 753.949444
  3. 2025-10-12T13:23:00 - RA: 351.615537, DEC: 9.239602, RA_rate: 1384.237500, DEC_rate: 412.990278
Generating ToO commands for observations...
sudo python make_too.py -ra '291.81307217' -dec '-12.73433584' -ra_rate '1643.18194444' -dec_rate '592.81222222' -obs_time_utc '2025-10-12T13:21:00' -filt 'r' -exp '10' -noexp '1' -name 'RISAT2BR2' -p '7000' -obs_type 'NS' -domesleep '50' -d '20251012' -iit_obs 'sameer' -hanle_obs 'rigzin' -stack 'N' -propnums 'P07'

sudo python make_too.py -ra '323.21988209' -dec '-0.75462156' -ra_rate '1938.50083333' -dec_rate '753.94944444' -obs_time_utc '2025-10-12T13:22:00' -filt 'r' -exp '10' -noexp '1' -name 'RISAT2BR2' -p '7000' -obs_type 'NS' -domesleep '50' -d '20251012' -iit_obs 'sam

In [4]:
# Save all commands to a file
output_file = '/Users/meghapatil/Documents/Sameer/Physics/vb/git/generated_too_commands.sh'

with open(output_file, 'w') as f:
    f.write("# Generated ToO Commands\n")
    f.write("# Generated on: " + datetime.now().strftime('%Y-%m-%d %H:%M:%S') + "\n")
    f.write(f"# Total commands: {len(all_commands)}\n")
    #f.write(f"# NOAA commands: {len(noaa_obs)}\n")
    #f.write(f"# NISAR commands: {len(nisar_obs)}\n\n")
    
    for i, command in enumerate(all_commands):
        f.write(f"# Command {i+1}\n")
        f.write(command + "\n\n")

print(f"All commands saved to: {output_file}")

# Summary statistics
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Total ToO commands generated: {len(all_commands)}")
#print(f"NOAA-19 commands: {len(noaa_obs)}")
#print(f"NISAR commands: {len(nisar_obs)}")
print(f"\nStatic parameters used:")
for key, value in static_params.items():
    print(f"  {key}: {value}")
print(f"\nRA_RATE and DEC_RATE conversion: Divided by 3600 (arcseconds/hour → degrees/hour)")
print(f"Observation types: 'NS' for non-sidereal tracking, 'S' for single exposure")
print(f"Output file: {output_file}")
print("="*60)


NameError: name 'all_commands' is not defined

In [2]:
import re
from datetime import datetime
from pathlib import Path
import glob
import numpy as np
# Static parameters (adjust if needed)
static_params = {
    'filt': 'r+r+r',
    'exp': '480+480+480',
    'noexp': '1+1+1',
    'name': '',
    'p': '5700',
    'obs_type': 'NS',
    'iit_obs': 'sameer',
    'hanle_obs': 'rigzin'
}

def load_template(path):
    text = Path(path).read_text().strip()
    # Ensure it's a single-line template
    text = " ".join(text.split())
    return text

def parse_line(line):
    # Example line format:
    # 2025 10 28 1100  21.1635    +14.549    107.3  19.9    0.30   0.070 290  +52   +10    0.38  047  +30   Map/Offsets
    cols = line.strip().split()
    if len(cols) < 8:
        return None

    # Date and UT (hm)
    year, month, day = cols[0], cols[1], cols[2]
    hm = cols[3]  # e.g., '1100' meaning 11:00:00
    if not re.fullmatch(r"\d{4}", hm):
        return None
    hh, mm = hm[:2], hm[2:]
    obs_dt = datetime.strptime(f"{year}-{month}-{day} {hh}:{mm}:00", "%Y-%m-%d %H:%M:%S")
    iso_time = obs_dt.strftime("%Y-%m-%dT%H:%M:%S")
    ymd = obs_dt.strftime("%Y%m%d")

    # RA in hours, DEC in degrees
    ra_deg = float(cols[4])*15
    dec_deg = float(cols[5])

    # Motion columns: header shows '"/sec  "/sec' so arcsec/sec.
    # make_too_v2.py commonly uses deg/hr; numerically, arcsec/sec equals deg/hr (factor 3600 cancels).
    ra_rate = float(cols[8]) 
    dec_rate = float(cols[9])

    # If signs are attached separately, handle them
    # Try to detect explicit sign tokens preceding numbers
    def signed_value(tokens, idx):
        if tokens[idx] in ["+", "-"]:
            return float(tokens[idx] + tokens[idx+1])
        return float(tokens[idx])

    try:
        # Try sign-aware parse using known approximate indices
        ra_rate = signed_value(cols, 8)
        dec_rate = signed_value(cols, 9)
    except Exception:
        # Fallback to previously parsed floats
        pass

    # Extract magnitude (V) from column 7
    mag = None
    if len(cols) > 7:
        try:
            mag = float(cols[7])
        except (ValueError, IndexError):
            pass

    return {
        "utc_time": iso_time,
        "date_ymd": ymd,
        "ra_deg": ra_deg,
        "dec_deg": dec_deg,
        "ra_rate": ra_rate,   # deg/hr numerically equal to arcsec/sec
        "dec_rate": dec_rate,  # deg/hr numerically equal to arcsec/sec
        "mag": mag
    }

def parse_ephem_file(path):
    lines = Path(path).read_text().splitlines()
    print(f'Number of lines in epherides = {len(lines)}')#\nSample:\n{lines[0]}')
    data = []
    # Skip first two header lines
    for i, line in enumerate(lines):
        print(line)
        if not line.strip():
            continue
        rec = parse_line(line)
        if rec:
            data.append(rec)
            print(f'Appending: {rec}')
    return data

def make_command(template, rec, params):
    cmd = template
    replacements = {
        "PH_RIGHT_ASC": f"{rec['ra_deg']:.8f}",
        "PH_DECLINATION": f"{rec['dec_deg']:.8f}",
        "PH_RATE_RA": f"{rec['ra_rate']:.8f}",
        "PH_RATE_DEC": f"{rec['dec_rate']:.8f}",
        "PH_OBS_TIME_UTC": rec["utc_time"],
        "PH_FILT": params["filt"],
        "PH_EXPOSURE": params["exp"],
        "PH_NAME": params["name"],
        "PH_PRIORITY": params["p"],
        "PH_OBS_TYPE": params["obs_type"],
        "PH_DATE": rec["date_ymd"],
        "PH_IIT_OBS": params["iit_obs"],
        "PH_HANLE_OBS": params["hanle_obs"],
        'PH_NO_EXP': static_params['noexp']
    }
    for k, v in replacements.items():
        cmd = cmd.replace(k, v)
    return cmd

def get_exp_from_magnitude(mag, mag_ref=20, exp_ref=600):
    """
    Calculate exposure time required to detect an object of given magnitude.
    
    Parameters:
    -----------
    mag : float
        Target magnitude
    mag_ref : float, optional
        Reference magnitude (default: 21)
    exp_ref : float, optional
        Reference exposure time in seconds for mag_ref (default: 600)
    
    Returns:
    --------
    float
        Required exposure time in seconds
    
    Formula:
    --------
    t = t_ref * 10^((mag - mag_ref) / 2.5)
    
    This follows the magnitude system where a difference of 2.5 magnitudes
    corresponds to a factor of 10 in brightness.
    """
    exp_time = exp_ref * (10 ** ((mag - mag_ref) / 2.5))
    return exp_time

print(get_exp_from_magnitude(21))
print(get_exp_from_magnitude(20))
print(get_exp_from_magnitude(19))
print(get_exp_from_magnitude(18))
print(get_exp_from_magnitude(17))
print('#'*20)
def get_exp(dec, ra_rate, dec_rate, mag, pix_scale=0.406, max_streak_length=50, exposure_limit=600, mag_ref=21, exp_ref=600):
    """
    Calculate exposure time considering both streak length and magnitude requirements.
    
    Parameters:
    -----------
    dec : float
        Declination in degrees
    ra_rate : float
        RA rate in deg/hr
    dec_rate : float
        DEC rate in deg/hr
    mag : float
        Target magnitude
    pix_scale : float, optional
        Pixel scale in arcseconds per pixel (default: 0.4)
    max_streak_length : float, optional
        Maximum streak length in arcseconds (default: 50)
    exposure_limit : float, optional
        Maximum exposure time limit in seconds (default: 600)
    mag_ref : float, optional
        Reference magnitude (default: 21)
    exp_ref : float, optional
        Reference exposure time in seconds for mag_ref (default: 600)
    
    Returns:
    --------
    float
        Required exposure time in seconds (maximum of streak-based and magnitude-based)
    """
    # Calculate exposure based on streak length
    net_rate = np.sqrt(dec_rate**2 + (ra_rate * np.cos(np.radians(dec))) ** 2)
    exp_time_streak = max_streak_length * pix_scale / net_rate 

    # Calculate exposure based on magnitude
    exp_time_mag = get_exp_from_magnitude(mag, mag_ref, exp_ref)

    print(f'dec={dec}, ra_rate={ra_rate}, dec_rate={dec_rate}, mag={mag}, net_rate={net_rate}, exp_time_streak={exp_time_streak}, exp_time_mag={exp_time_mag}')
    if exp_time_mag < exp_time_streak: return np.minimum(exp_time_streak, exposure_limit)
    else: return -1

TEMPLATE_FILE = "/Users/meghapatil/Documents/Sameer/Physics/vb/git/asteroids/maketoo_template"
date = datetime.now().strftime('%y%m%d')
#date = '251128'
TODAYS_DIR = f"/Users/meghapatil/Documents/Sameer/Physics/vb/git/asteroids/{date}"
TODAYS_FILE = f"{TODAYS_DIR}/{date}.txt"
OUTPUT_FILE = f"{TODAYS_DIR}/generated_too_commands.sh"
template = load_template(TEMPLATE_FILE)
print(f'TODAYS_DIR={TODAYS_DIR}')
print(f'OUTPUT_FILE={OUTPUT_FILE}')

# Function to check if a line contains a single word with 7 alphanumeric characters (object name)
def is_object_name(line):
    words = line.strip().split()
    if len(words) == 1:
        word = words[0]
        # Check if it's exactly 7 alphanumeric characters
        if len(word) == 7 and word.isalnum():
            return True
    return False

# Read TODAYS_FILE line by line and process objects
name = None
ephemeris_lines = []
all_commands = []

with open(TODAYS_FILE, 'r') as f:
    for line in f:
        line_stripped = line.strip()
        
        # Check if this line is an object name
        if is_object_name(line_stripped):
            # If we have a previous object, process its ephemerides
            if name is not None and ephemeris_lines:
                print(f'## {name} ##')
                print('#############')
                static_params['name'] = name
                
                # Parse all ephemeris lines for this object
                obs_list = []
                for ephem_line in ephemeris_lines:
                    rec = parse_line(ephem_line)
                    if rec:
                        obs_list.append(rec)
                
                if obs_list:
                    # Take the middle ephemeris line
                    middle_idx = len(obs_list) // 2
                    rec = obs_list[middle_idx]
                    
                    # Calculate exposure time (use magnitude if available, otherwise default to 20)
                    mag = rec.get('mag', 20)
                    exp_time = get_exp(dec=rec['dec_deg'], ra_rate=rec['ra_rate'], dec_rate=rec['dec_rate'], mag=mag)
                    
                    if exp_time > 0:
                        # Update static_params with calculated exposure time
                        static_params['exp'] = f'{int(exp_time)}+{int(exp_time)}+{int(exp_time)}'
                        cmd = make_command(template, rec, static_params)
                        all_commands.append(cmd)
                        print(cmd)
                    else:
                        print(f'Skipping due to exp time: {rec}')
                else:
                    print("No observations parsed.")
            
            # Set new object name and reset ephemeris lines
            name = line_stripped
            ephemeris_lines = []
        else:
            # This is not an object name line
            # If we have a current object, check if this is an ephemeris line
            if name is not None:
                rec = parse_line(line_stripped)
                if rec:
                    # This is a valid ephemeris line, add it to the list
                    ephemeris_lines.append(line_stripped)

# Process the last object if there is one
if name is not None and ephemeris_lines:
    print(f'## {name} ##')
    print('#############')
    static_params['name'] = name
    
    # Parse all ephemeris lines for this object
    obs_list = []
    for ephem_line in ephemeris_lines:
        rec = parse_line(ephem_line)
        if rec:
            obs_list.append(rec)
    
    if obs_list:
        # Take the middle ephemeris line
        middle_idx = len(obs_list) // 2
        rec = obs_list[middle_idx]
        
        # Calculate exposure time (use magnitude if available, otherwise default to 20)
        mag = rec.get('mag', 20)
        exp_time = get_exp(dec=rec['dec_deg'], ra_rate=rec['ra_rate'], dec_rate=rec['dec_rate'], mag=mag)
        
        if exp_time > 0:
            # Update static_params with calculated exposure time
            static_params['exp'] = f'{int(exp_time)}+{int(exp_time)}+{int(exp_time)}'
            cmd = make_command(template, rec, static_params)
            all_commands.append(cmd)
            print(cmd)
        else:
            print(f'Skipping due to exp time: {rec}')
    else:
        print("No observations parsed.")

# Write all commands to output file
with open(OUTPUT_FILE, 'w') as f:
    f.write("\n".join(all_commands) + "\n")

print(f"\nTotal commands generated: {len(all_commands)}")
print(f"Commands written to: {OUTPUT_FILE}")


1507.131858905748
600.0
238.86430233209833
95.0935915476668
37.8574406688116
####################
TODAYS_DIR=/Users/meghapatil/Documents/Sameer/Physics/vb/git/asteroids/251205
OUTPUT_FILE=/Users/meghapatil/Documents/Sameer/Physics/vb/git/asteroids/251205/generated_too_commands.sh
## RdSH484 ##
#############
dec=30.453, ra_rate=-0.021, dec_rate=0.003, mag=19.5, net_rate=0.018349843998734912, exp_time_streak=1106.2764349059064, exp_time_mag=150.7131858905748
sudo python make_too.py -ra '87.68400000' -dec '30.45300000' -ra_rate '-0.02100000' -dec_rate '0.00300000' -obs_time_utc '2025-12-05T18:50:00' -filt 'r+r+r' -exp '600+600+600' -noexp '1+1+1' -name 'RdSH484' -p '5700' -obs_type 'NS' -iit_obs 'sameer' -pi 'sameer' -hanle_obs 'rigzin' -stack 'N' -propnums 'P07'
## orA0886 ##
#############
dec=25.36, ra_rate=-0.016, dec_rate=0.024, mag=19.5, net_rate=0.028018532639047345, exp_time_streak=724.520454426275, exp_time_mag=150.7131858905748
sudo python make_too.py -ra '101.35200000' -dec '25.